In [46]:
import sys,os
sys.path.append(os.path.realpath('..'))

from scripts.functions import dilute, replenish,dilute_species, replenish_species, run_model,plot_result
from scripts.species import DNA, RNA, Protein, Resource
from scripts.models import Repressor_model


import numpy as np
import scipy.integrate
import scipy.optimize
from bokeh.plotting import figure, output_file, show

import bokeh.plotting
import bokeh.io
from bokeh.models import Span


# We will use Matplotlib to make a 3D plot
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import panel as pn
pn.extension()

bokeh.io.output_notebook()

Loading BokehJS ...

In [89]:
#######Define species #######

#T7 RNAP
T7_DNA = DNA()
T7_RNA = RNA()
T7 = Protein()

#Repressor
Repressor_DNA = DNA()
Repressor_RNA = RNA()
Repressor = Protein()

#GFP
GFP_DNA = DNA()
GFP_RNA = RNA()
GFP = Protein()

#Resource
R      = Resource()

#Others
TX = 1
TL = 1

#list of molecules
molecules_list = [T7_DNA,T7_RNA,T7,GFP_DNA,GFP_RNA,GFP,Repressor_DNA,Repressor_RNA,Repressor,R]


#give index num
for i in range(0,len(molecules_list)):
    molecules_list[i].idx = i




#define values 
##T7
T7_DNA.ic =0.01
T7_DNA.lc =0.01

T7.ic = 1
T7.lc = 1

##GFP
GFP_DNA.ic = 2
GFP_DNA.lc = 2

GFP.lc = 0

##Repressor 
Repressor_DNA.ic = 0
Repressor_DNA.lc = 0


#resources
R.ic =100
R.lc =100

#Define parameters
alpha=0.7;
beta=0.07;
K=1.0;
Kd = 0.5;
HC = 1;
KTL=10;
KTX=1;
lamb1=0;
lamb2=0;

parameters_list = (alpha,beta,K,Kd,KTL,KTX,lamb1,lamb2)



In [90]:
def run_model(model,t,parameters_list,molecules_0,dilute_list,replenish_list,result_all):  
    start_cycle,end_cycle = np.array(t)*4
    for n in range (start_cycle,end_cycle):
        #define time
        t_start= n*15
        t_end = (n+1)*15
        t= np.linspace(t_start,t_end,2)
                
        #dilution 
        ###diute out
        dilute_species((dilute_list),molecules_0)
        
        ###replenish 
        replenish_species((replenish_list),molecules_0)
        result_all = np.append(result_all,molecules_0)

        #solve equation and save result
        result = scipy.integrate.odeint(model, molecules_0, t, args=parameters_list)
        
        #update parameter
        molecules_0 = result.transpose()[:,-1]

    return result_all,molecules_0
    

In [91]:
#Positive control
T7_DNA.ic = 0
T7_DNA.lc = 0

#initial value
molecules_0 = [None]*len(molecules_list) #create empty list


for i in range(0,len(molecules_list)):
    molecules_0[i] = molecules_list[i].ic

#stage 1 kick start
result_all=[molecules_0] #initial condition


#kick start
dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
replenish_list = [T7,R]
result_all,molecules_0 = run_model(Repressor_model,(0,4),parameters_list,molecules_0,dilution_list,replenish_list,result_all)


#self regeneration 
dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
replenish_list = [T7,R]
result_all,molecules_0 = run_model(Repressor_model,(4,16),parameters_list,molecules_0,dilution_list,replenish_list,result_all)



#wash out
dilution_list = [T7_DNA,T7_RNA,T7,GFP,GFP_RNA,Repressor_DNA,Repressor,Repressor_RNA,R]
replenish_list = [T7,R]

result_all,molecules_0=run_model(Repressor_model,(16,30),parameters_list,molecules_0,dilution_list,replenish_list,result_all)



result_all = np.resize(result_all,(121,10))
for i in range(0,len(molecules_list)):
        (molecules_list[i]).pc_result = result_all.transpose()[i]

#T7_DNA,T7_RNA,T7,GFP_DNA,GFP_RNA,GFP,R = result_all.transpose()
plot_result(GFP.pc_result)

In [92]:
#Negative control
T7_DNA.ic = 0
T7_DNA.lc = 0

#initial value
molecules_0 = [None]*len(molecules_list) #create empty list


for i in range(0,len(molecules_list)):
    molecules_0[i] = molecules_list[i].ic

#stage 1 kick start
result_all=[molecules_0] #initial condition


#kick start
dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
replenish_list = [T7,R]
result_all,molecules_0 = run_model(Repressor_model,(0,4),parameters_list,molecules_0,dilution_list,replenish_list,result_all)


#self regeneration 
dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
replenish_list = [R]
result_all,molecules_0 = run_model(Repressor_model,(4,16),parameters_list,molecules_0,dilution_list,replenish_list,result_all)



#wash out
dilution_list = [T7_DNA,T7_RNA,T7,GFP,GFP_RNA,Repressor_DNA,Repressor,Repressor_RNA,R]
replenish_list = [R]

result_all,molecules_0=run_model(Repressor_model,(16,30),parameters_list,molecules_0,dilution_list,replenish_list,result_all)



result_all = np.resize(result_all,(121,10))
for i in range(0,len(molecules_list)):
        (molecules_list[i]).nc_result = result_all.transpose()[i]

#T7_DNA,T7_RNA,T7,GFP_DNA,GFP_RNA,GFP,R = result_all.transpose()
plot_result(T7.nc_result)

In [93]:
#experiment 
T7_DNA.ic = 0.001
T7_DNA.lc = 0.001
#initial value
molecules_0 = [None]*len(molecules_list) #create empty list

for i in range(0,len(molecules_list)):
    molecules_0[i] = molecules_list[i].ic

#stage 1 kick start
result_all=[molecules_0] #initial condition


#DR = 0.2

#kick start
dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
replenish_list = [T7,R]
result_all,molecules_0 = run_model(Repressor_model,(0,4),parameters_list,molecules_0,dilution_list,replenish_list,result_all)


#self regeneration 
dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
replenish_list = [R]
result_all,molecules_0 = run_model(Repressor_model,(4,16),parameters_list,molecules_0,dilution_list,replenish_list,result_all)



#wash out
dilution_list = [T7_DNA,T7_RNA,T7,GFP,GFP_RNA,Repressor_DNA,Repressor,Repressor_RNA,R]
replenish_list = [R]

result_all,molecules_0=run_model(Repressor_model,(16,30),parameters_list,molecules_0,dilution_list,replenish_list,result_all)



result_all = np.resize(result_all,(121,10))
for i in range(0,len(molecules_list)):
        (molecules_list[i]).result = result_all.transpose()[i]

#T7_DNA,T7_RNA,T7,GFP_DNA,GFP_RNA,GFP,R = result_all.transpose()
GFP.result1 = GFP.result

In [94]:
#experiment 
T7_DNA.ic = 0.01
T7_DNA.lc = 0.01
#initial value
molecules_0 = [None]*len(molecules_list) #create empty list

for i in range(0,len(molecules_list)):
    molecules_0[i] = molecules_list[i].ic

#stage 1 kick start
result_all=[molecules_0] #initial condition


#DR = 0.2

#kick start
dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
replenish_list = [T7,R]
result_all,molecules_0 = run_model(Repressor_model,(0,4),parameters_list,molecules_0,dilution_list,replenish_list,result_all)


#self regeneration 
dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
replenish_list = [R]
result_all,molecules_0 = run_model(Repressor_model,(4,16),parameters_list,molecules_0,dilution_list,replenish_list,result_all)



#wash out
dilution_list = [T7_DNA,T7_RNA,T7,GFP,GFP_RNA,Repressor_DNA,Repressor,Repressor_RNA,R]
replenish_list = [R]

result_all,molecules_0=run_model(Repressor_model,(16,30),parameters_list,molecules_0,dilution_list,replenish_list,result_all)



result_all = np.resize(result_all,(121,10))
for i in range(0,len(molecules_list)):
        (molecules_list[i]).result = result_all.transpose()[i]

#T7_DNA,T7_RNA,T7,GFP_DNA,GFP_RNA,GFP,R = result_all.transpose()
GFP.result2 = GFP.result

In [95]:
#experiment 
T7_DNA.ic = 1
T7_DNA.lc = 1
#initial value
molecules_0 = [None]*len(molecules_list) #create empty list

for i in range(0,len(molecules_list)):
    molecules_0[i] = molecules_list[i].ic

#stage 1 kick start
result_all=[molecules_0] #initial condition


#DR = 0.2

#kick start
dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
replenish_list = [T7,R]
result_all,molecules_0 = run_model(Repressor_model,(0,4),parameters_list,molecules_0,dilution_list,replenish_list,result_all)


#self regeneration 
dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
replenish_list = [R]
result_all,molecules_0 = run_model(Repressor_model,(4,16),parameters_list,molecules_0,dilution_list,replenish_list,result_all)



#wash out
dilution_list = [T7_DNA,T7_RNA,T7,GFP,GFP_RNA,Repressor_DNA,Repressor,Repressor_RNA,R]
replenish_list = [R]

result_all,molecules_0=run_model(Repressor_model,(16,30),parameters_list,molecules_0,dilution_list,replenish_list,result_all)



result_all = np.resize(result_all,(121,10))
for i in range(0,len(molecules_list)):
        (molecules_list[i]).result = result_all.transpose()[i]

#T7_DNA,T7_RNA,T7,GFP_DNA,GFP_RNA,GFP,R = result_all.transpose()
GFP.result3 = GFP.result

In [96]:

t = np.linspace(0, 15*(len(GFP.pc_result)-1), len(GFP.pc_result))
p = bokeh.plotting.figure(
    plot_width=2000,
    plot_height=500,
    x_axis_label="time(h)",
    y_axis_label="GFP",
    y_axis_type="linear",
    y_range=(0, 80)
)

colors = bokeh.palettes.d3["Category10"][3]

# Populate glyphs
p.line(t/60, GFP.pc_result[1:], line_width=4, color=colors[2])
p.line(t/60, GFP.nc_result[1:], line_width=4, color=colors[1])
p.line(t/60, GFP.result3[1:], line_width=4, color=colors[0],alpha = 1)
p.line(t/60, GFP.result2[1:], line_width=4, color=colors[0],alpha = 0.6)
p.line(t/60, GFP.result1[1:], line_width=4, color=colors[0],alpha = 0.3)
vline1 = Span(location=4, dimension='height', line_color='black', line_width=1,line_dash='dashed')
vline2 = Span(location=16, dimension='height', line_color='black', line_width=1,line_dash='dashed')
p.add_layout(vline1)
p.add_layout(vline2)
show(p)


In [82]:
GFP.result3

array([ 0.        ,  0.        , 11.73510752, 11.93864038, 12.08655642,
       12.19867645, 12.2871914 , 12.35961381, 12.42058467, 12.47299111,
       12.5186586 , 12.55877731, 12.59416184, 12.6254068 , 12.65297801,
       12.67726446, 12.69860687, 12.71731263, 12.7832939 , 12.87085852,
       12.96438963, 13.05502465, 13.13822478, 13.21209657, 13.27628324,
       13.33125955, 13.37789915, 13.41721981, 13.45024102, 13.47791074,
       13.50107328, 13.52046057, 13.53669606, 13.55030468, 13.56172504,
       13.57132196, 13.57939805, 13.58620406, 13.59194774, 13.59680136,
       13.60090798, 13.60438655, 13.60733622, 13.60983979, 13.61196652,
       13.6137745 , 13.61531253, 13.61662169, 13.61773661, 13.61868654,
       13.61949621, 13.62018657, 13.62077536, 13.62127766, 13.62170627,
       13.62207207, 13.62238431, 13.62265088, 13.62287847, 13.62307281,
       13.62323877, 13.62338051, 13.62350156, 13.62360496, 13.62369328,
       13.62376872, 13.77788345, 14.06424682, 14.47297538, 14.99